In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import  pickle
import ast
import os
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Download if not already done
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vasee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vasee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vasee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vasee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
#Load data
movies = pd.read_csv("../data/tmdb_5000_movies.csv")
credits = pd.read_csv("../data/tmdb_5000_credits.csv")

In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
credits = credits.rename(columns={"movie_id": "id"})

In [8]:
all_data = movies.merge(credits, on = "id")
all_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [9]:
all_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

In [10]:
selected_data = all_data[['genres', 'id', 'keywords','original_title', 'overview','vote_average','cast', 'crew']]

In [11]:
selected_data.isnull().sum()

genres            0
id                0
keywords          0
original_title    0
overview          3
vote_average      0
cast              0
crew              0
dtype: int64

In [12]:
selected_data.dropna(inplace = True)

C:\Users\vasee\AppData\Local\Temp\ipykernel_17908\1532868584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.dropna(inplace = True)


In [13]:
selected_data.head(3)

,genres,id,keywords,original_title,overview,vote_average,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",7.2,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",6.9,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,6.3,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


## Converting string to list

In [14]:
selected_data.genres[1]

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

In [15]:
def str_to_list(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [16]:
selected_data.loc[:,"genres"] = selected_data.loc[:,"genres"].apply(str_to_list)

In [17]:
selected_data.keywords[1]

'[{"id": 270, "name": "ocean"}, {"id": 726, "name": "drug abuse"}, {"id": 911, "name": "exotic island"}, {"id": 1319, "name": "east india trading company"}, {"id": 2038, "name": "love of one\'s life"}, {"id": 2052, "name": "traitor"}, {"id": 2580, "name": "shipwreck"}, {"id": 2660, "name": "strong woman"}, {"id": 3799, "name": "ship"}, {"id": 5740, "name": "alliance"}, {"id": 5941, "name": "calypso"}, {"id": 6155, "name": "afterlife"}, {"id": 6211, "name": "fighter"}, {"id": 12988, "name": "pirate"}, {"id": 157186, "name": "swashbuckler"}, {"id": 179430, "name": "aftercreditsstinger"}]'

In [18]:
selected_data.loc[:,"keywords"] = selected_data.loc[:,"keywords"].apply(str_to_list)

In [19]:
selected_data.cast[1]

'[{"cast_id": 4, "character": "Captain Jack Sparrow", "credit_id": "52fe4232c3a36847f800b50d", "gender": 2, "id": 85, "name": "Johnny Depp", "order": 0}, {"cast_id": 5, "character": "Will Turner", "credit_id": "52fe4232c3a36847f800b511", "gender": 2, "id": 114, "name": "Orlando Bloom", "order": 1}, {"cast_id": 6, "character": "Elizabeth Swann", "credit_id": "52fe4232c3a36847f800b515", "gender": 1, "id": 116, "name": "Keira Knightley", "order": 2}, {"cast_id": 12, "character": "William \\"Bootstrap Bill\\" Turner", "credit_id": "52fe4232c3a36847f800b52d", "gender": 2, "id": 1640, "name": "Stellan Skarsg\\u00e5rd", "order": 3}, {"cast_id": 10, "character": "Captain Sao Feng", "credit_id": "52fe4232c3a36847f800b525", "gender": 2, "id": 1619, "name": "Chow Yun-fat", "order": 4}, {"cast_id": 9, "character": "Captain Davy Jones", "credit_id": "52fe4232c3a36847f800b521", "gender": 2, "id": 2440, "name": "Bill Nighy", "order": 5}, {"cast_id": 7, "character": "Captain Hector Barbossa", "credit_

In [20]:
selected_data.crew[1]

'[{"credit_id": "52fe4232c3a36847f800b579", "department": "Camera", "gender": 2, "id": 120, "job": "Director of Photography", "name": "Dariusz Wolski"}, {"credit_id": "52fe4232c3a36847f800b4fd", "department": "Directing", "gender": 2, "id": 1704, "job": "Director", "name": "Gore Verbinski"}, {"credit_id": "52fe4232c3a36847f800b54f", "department": "Production", "gender": 2, "id": 770, "job": "Producer", "name": "Jerry Bruckheimer"}, {"credit_id": "52fe4232c3a36847f800b503", "department": "Writing", "gender": 2, "id": 1705, "job": "Screenplay", "name": "Ted Elliott"}, {"credit_id": "52fe4232c3a36847f800b509", "department": "Writing", "gender": 2, "id": 1706, "job": "Screenplay", "name": "Terry Rossio"}, {"credit_id": "52fe4232c3a36847f800b57f", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "52fe4232c3a36847f800b585", "department": "Editing", "gender": 2, "id": 1722, "job": "Editor", "name": "Craig Wood"}, {"credit_id": "52f

In [21]:
def convert_cast(text):
    counter = 0
    l = []
    for i in ast.literal_eval(text):
        if counter < 5:
            l.append(i['name'])
        counter += 1           
    return l

In [22]:
selected_data.loc[:,"cast"] = selected_data.loc[:,"cast"].apply(convert_cast)

In [23]:
def get_director(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [24]:
selected_data.loc[:,"crew"] = selected_data.loc[:,"crew"].apply(get_director)

In [25]:
selected_data.loc[:,"overview"] = selected_data.loc[:,"overview"].apply(lambda x: x.split())

In [26]:
selected_data.sample(4)

,genres,id,keywords,original_title,overview,vote_average,cast,crew
4233,"[Thriller, Action, Horror]",323271,[],Unsullied,"[When, car, trouble, strands, track, star, Rea...",3.0,"[Rusty Joiner, Murray Gray, James Gaudioso, Er...",[Simeon Rice]
2901,"[War, Drama]",50601,"[journalist, interpreter, georgia europe, war ...",5 Days of War,"[An, American, journalist, and, his, cameraman...",5.8,"[Rupert Friend, Val Kilmer, Andy García, Dean ...",[Renny Harlin]
1432,"[Action, Comedy, Crime]",268920,"[texas, prisoner, cop, drug cartel, on the run...",Hot Pursuit,"[An, uptight, by-the-book, cop, must, protect,...",5.4,"[Reese Witherspoon, Sofía Vergara, Michael Mos...",[Anne Fletcher]
4718,[Comedy],18632,[independent film],A Dog's Breakfast,"[If, you've, never, been, good, at, anything, ...",5.9,"[David Hewlett, Kate Hewlett, Paul McGillion, ...",[David Hewlett]


## Remove Space

In [27]:
# now removing space like that 
'Anna Kendrick'
'Anna_Kendrick'

def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ","_"))
    return L1

In [28]:
selected_data.loc[:,'cast'] = selected_data.loc[:,'cast'].apply(remove_space)
selected_data.loc[:,'crew'] = selected_data.loc[:,'crew'].apply(remove_space)
selected_data.loc[:,'genres'] = selected_data.loc[:,'genres'].apply(remove_space)
selected_data.loc[:,'keywords'] = selected_data.loc[:,'keywords'].apply(remove_space)

## Creating tags

In [29]:
selected_data['tags'] = selected_data['overview'] + selected_data['genres'] + selected_data['keywords'] + selected_data['cast'] + selected_data['crew']


C:\Users\vasee\AppData\Local\Temp\ipykernel_17908\2434083119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['tags'] = selected_data['overview'] + selected_data['genres'] + selected_data['keywords'] + selected_data['cast'] + selected_data['crew']


In [30]:
selected_data.head()

,genres,id,keywords,original_title,overview,vote_average,cast,crew,tags
0,"[Action, Adventure, Fantasy, Science_Fiction]",19995,"[culture_clash, future, space_war, space_colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",7.2,"[Sam_Worthington, Zoe_Saldana, Sigourney_Weave...",[James_Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug_abuse, exotic_island, east_india_...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",6.9,"[Johnny_Depp, Orlando_Bloom, Keira_Knightley, ...",[Gore_Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]",206647,"[spy, based_on_novel, secret_agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...",6.3,"[Daniel_Craig, Christoph_Waltz, Léa_Seydoux, R...",[Sam_Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc_comics, crime_fighter, terrorist, secret_i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",7.6,"[Christian_Bale, Michael_Caine, Gary_Oldman, A...",[Christopher_Nolan],"[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, Science_Fiction]",49529,"[based_on_novel, mars, medallion, space_travel...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...",6.1,"[Taylor_Kitsch, Lynn_Collins, Samantha_Morton,...",[Andrew_Stanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [31]:
final_df = selected_data[["id","original_title","tags","vote_average"]]

In [32]:
final_df.head(2)

,id,original_title,tags,vote_average
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",7.2
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",6.9


In [33]:
final_df['tags'] = final_df['tags'].apply(lambda x: ",".join(x))

C:\Users\vasee\AppData\Local\Temp\ipykernel_17908\3139569575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tags'] = final_df['tags'].apply(lambda x: ",".join(x))


In [34]:
final_df['tags'] = final_df['tags'].apply(lambda x: x.lower().replace(",", " ").replace(".", " "))

C:\Users\vasee\AppData\Local\Temp\ipykernel_17908\4125928009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tags'] = final_df['tags'].apply(lambda x: x.lower().replace(",", " ").replace(".", " "))


In [35]:
final_df.head(2)

,id,original_title,tags,vote_average
0,19995,Avatar,in the 22nd century a paraplegic marine is di...,7.2
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead ha...,6.9


In [36]:
final_df.tags[1]

"captain barbossa  long believed to be dead  has come back to life and is headed to the edge of the earth with will turner and elizabeth swann  but nothing is quite as it seems  adventure fantasy action ocean drug_abuse exotic_island east_india_trading_company love_of_one's_life traitor shipwreck strong_woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnny_depp orlando_bloom keira_knightley stellan_skarsgård chow_yun-fat gore_verbinski"

## Lemmatization

In [37]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

def lemmatize_text_better(text):
    words = text.split()
    pos_tags = pos_tag(words)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return " ".join(lemmatized)

# Apply to your dataframe
final_df['tags'] = final_df['tags'].apply(lemmatize_text_better)

C:\Users\vasee\AppData\Local\Temp\ipykernel_17908\2652589454.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tags'] = final_df['tags'].apply(lemmatize_text_better)


In [38]:
final_df.to_csv("../data/final_df.csv")

In [105]:
final_df.tags[1]

"captain barbossa long believe to be dead have come back to life and be head to the edge of the earth with will turner and elizabeth swann but nothing be quite a it seem adventure fantasy action ocean drug_abuse exotic_island east_india_trading_company love_of_one's_life traitor shipwreck strong_woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnny_depp orlando_bloom keira_knightley stellan_skarsgård chow_yun-fat gore_verbinski"

## Vectorization

In [106]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(final_df['tags']).toarray()

In [107]:
vector.shape

(4800, 5000)

In [108]:
similarity = cosine_similarity(vector)
similarity.shape

(4800, 4800)

## Recommender funtion

In [109]:
def recommender(movie):
    index = final_df[final_df['original_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(final_df.iloc[i[0]].original_title)

In [110]:
recommender("Batman")

Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns
Batman Forever


## Saving

In [111]:
os.makedirs("../output",exist_ok=True)

pickle.dump(final_df,open('../output/movie_list.pkl','wb'))
pickle.dump(similarity,open('../output/similarity.pkl','wb'))